In [1]:
import pandas as pd
import requests
from datetime import datetime

In [2]:
# Montgomery County, Maryland coordinates
latitude = 39.1547
longitude = -77.2405


In [3]:
filePath = '../Resources/Montgomery_Crash_Data.csv'
df = pd.read_csv(filePath)

In [4]:
df.columns

Index(['Unnamed: 0', 'index', 'Report Number', 'Crash Date/Time', 'Weather',
       'Surface Condition', 'Injury Severity', 'Vehicle ID', 'Vehicle Year',
       'Vehicle Make', 'Vehicle Model', 'Latitude', 'Longitude', 'geometry'],
      dtype='object')

In [5]:
df_dateTimeSeverity = df[['Crash Date/Time', 'Injury Severity']]
df_dateTimeSeverity.head()

,Crash Date/Time,Injury Severity
0,2021-04-04 12:00:00,NO APPARENT INJURY
1,2023-12-04 21:37:00,SUSPECTED MINOR INJURY
2,2024-07-17 21:27:00,SUSPECTED MINOR INJURY
3,2016-06-21 09:49:00,NO APPARENT INJURY
4,2016-12-05 06:55:00,SUSPECTED MINOR INJURY


In [6]:
unique_severity = df_dateTimeSeverity['Injury Severity'].unique()
display(unique_severity)

array(['NO APPARENT INJURY', 'SUSPECTED MINOR INJURY', 'FATAL INJURY',
       'SUSPECTED SERIOUS INJURY'], dtype=object)

In [7]:
severity_count = df_dateTimeSeverity['Injury Severity'].value_counts()
display(severity_count)

Injury Severity
NO APPARENT INJURY          86159
SUSPECTED MINOR INJURY      16779
SUSPECTED SERIOUS INJURY      936
FATAL INJURY                  128
Name: count, dtype: int64

In [8]:
df_dateTimeSeverity = df_dateTimeSeverity.sort_values(by='Crash Date/Time')
df_dateTimeSeverity.reset_index(drop=True, inplace=True)
df_dateTimeSeverity.head()

,Crash Date/Time,Injury Severity
0,2015-01-01 00:30:00,NO APPARENT INJURY
1,2015-01-01 01:10:00,NO APPARENT INJURY
2,2015-01-01 01:14:00,NO APPARENT INJURY
3,2015-01-01 01:26:00,NO APPARENT INJURY
4,2015-01-01 02:00:00,NO APPARENT INJURY


In [9]:
df_dateTimeSeverity.dtypes


Crash Date/Time    object
Injury Severity    object
dtype: object

In [10]:
df['Crash Date/Time'] = pd.to_datetime(df['Crash Date/Time'])
df_dateTimeSeverity.dtypes


Crash Date/Time    object
Injury Severity    object
dtype: object

In [11]:
def get_sun_times(date):
    api_url = f"https://api.sunrise-sunset.org/json?lat={latitude}&lng={longitude}&date={date}&formatted=0"
    print(api_url)
    response = requests.get(api_url)
    data = response.json()['results']
    
    sunrise = datetime.fromisoformat(data['sunrise']).time()
    sunset = datetime.fromisoformat(data['sunset']).time()
    civil_twilight_begin = datetime.fromisoformat(data['civil_twilight_begin']).time()  # Dawn
    civil_twilight_end = datetime.fromisoformat(data['civil_twilight_end']).time()  # Dusk
    
    return {
        'sunrise': sunrise,
        'sunset': sunset,
        'dawn': civil_twilight_begin,
        'dusk': civil_twilight_end
    }


In [12]:
def categorize_time(time, sun_times):
    time_only = time.time()
    if sun_times['sunrise'] <= time_only < sun_times['sunset']:
        return 'day'
    elif sun_times['dawn'] <= time_only < sun_times['sunrise']:
        return 'dawn'
    elif sun_times['sunset'] <= time_only < sun_times['dusk']:
        return 'dusk'
    else:
        return 'night'

In [13]:
# df_dateTimeSeverity['Crash Date/Time'] = df_dateTimeSeverity['Crash Date/Time'].dt.date

# Create a dictionary to store sunrise/sunset times for each unique date
sun_times_dict = {}

# Loop through unique dates to get sun times from the API
for date in df_dateTimeSeverity['Crash Date/Time'].unique():
    sun_times_dict[date] = get_sun_times(date)

# Now, apply the time categorization for each row
# df_dateTimeSeverity['time_category'] = df_dateTimeSeverity.apply(lambda row: categorize_time(row['time'], sun_times_dict[row['date']]), axis=1)

# # Print the DataFrame with the new 'time_category' column
# print(df_dateTimeSeverity[['time', 'time_category']])


https://api.sunrise-sunset.org/json?lat=39.1547&lng=-77.2405&date=2015-01-01 00:30:00&formatted=0


JSONDecodeError: Expecting value: line 1 column 1 (char 0)